In [2]:
import ee
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [4]:
def dms_to_decimal(degrees, minutes, seconds, direction):
    dd = degrees + (minutes / 60.0) + (seconds / 3600.0)
    if direction in ['S', 'W']:  # South and West are negative
        dd *= -1
    return dd

In [ ]:
# Example DMS coordinates of the four corners (Replace with actual values)
corners_dms = [
    ((37, 39.267, 0), (119, 41.152, 0), 'N', 'E'),  # Top-left
    ((37, 39.266, 0), (119, 40.350, 0), 'N', 'E'), # Top-right
    ((37, 38.564, 0), (119, 40.349, 0), 'N', 'E'), # Bottom-right
    ((37, 38.565, 0), (119, 41.158, 0), 'N', 'E')   # Bottom-left
]

# Convert DMS to Decimal Degrees
corner_points = [
    [dms_to_decimal(*lat_dms, lat_dir), dms_to_decimal(*lon_dms, lon_dir)]
    for (lat_dms, lon_dms, lat_dir, lon_dir) in corners_dms
]

# Ensure polygon closes by repeating the first point at the end
corner_points.append(corner_points[0])

In [ ]:
# Define Area of Interest (AOI)
aoi = ee.Geometry.Point([corner_points[0][1], corner_points[0][0]]).buffer(1000).bounds()

# Load Sentinel-2 imagery (replace with actual time period)
sentinel2_image = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(aoi) \
    .filterDate('2024-12-25', '2024-12-27') \
    .median()  # Taking median composite over the time period

# Select Bands 3 (Green) and 4 (Red)
green_band = sentinel2_image.select('B3')  # Green band (560 nm)
red_band = sentinel2_image.select('B4')    # Red band (665 nm)

# Calculate Chlorophyll Index (CI)
chlorophyll_index = green_band.subtract(red_band).divide(green_band.add(red_band))

# Visualize Chlorophyll Index
chlorophyll_index_vis = chlorophyll_index.clip(aoi)
chlorophyll_index_map = chlorophyll_index_vis.getMapId()



In [12]:
import folium
# Define visualization parameters
vis_params = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Get map visualization ID and token for Chlorophyll Index (CI)
map_id = chlorophyll_index.getMapId(vis_params)

# Create a folium map to display the result
my_map = folium.Map(location=[corner_points[0][0], corner_points[0][1]], zoom_start=12)
folium.TileLayer(
    tiles=map_id['tileUrl'],
    attr='Google Earth Engine',
    overlay=True,
    control=True
).add_to(my_map)

# Display the map
my_map

KeyError: 'tileUrl'